# Import packages





In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import anndata as ad
import scanpy as sc
import seaborn as sns
import squidpy as sq
from matplotlib_scalebar.scalebar import ScaleBar
from tqdm import tqdm
from skimage.measure import block_reduce
import scipy
import tifffile
import string

# Plot the spatial expression of LR pairs based on results from NICHES.R

In [ ]:
#Load in RCTD cell type info
meta = pd.read_csv('Puck_RCTD_Cell_Type_and_Coords_Weights.csv',header=0, index_col = 0)
coords = meta.drop(columns=['CellType'])
print(coords.shape)
coords.head(3)

In [ ]:
#Load in the LR pair expression data generated by running NICHES.R
niche = pd.read_csv('puck_niche_signaling_raw_data.csv',header=0, index_col = 0)
niche_t = niche.T
niche_t = niche_t.reset_index()
niche_t = niche_t.rename(columns={'index': 'barcode'})
niche_t = niche_t.set_index('barcode')
print(niche_t.shape)
niche_t.head(3)

In [ ]:
#Generate anndata object
genes = niche_t.columns.values
counts_sparse = scipy.sparse.csr_matrix(niche_t.to_numpy())
counts_sparse.eliminate_zeros()
adata = ad.AnnData(counts_sparse, obs=coords)
adata.obs_names = coords.barcode.values
adata.var_names = genes
adata.obsm["spatial"] = coords[coords.barcode.isin(adata.obs_names.values)][["x", "y"]].to_numpy()
adata.obs["cluster"] = meta.CellType.values

In [ ]:
#Calculate Moran's I statistics
sq.gr.spatial_neighbors(adata, coord_type="generic")
sq.gr.spatial_autocorr(adata, mode="moran")
adata.uns["moranI"].to_csv('Puck_Niche_Signaling_Morans_I.csv')
adata.uns["moranI"].head(10)

In [ ]:
#Plot LR pair
sc.pl.spatial(
    adata,
    vmax=40,
    color=['LR pair of interests'],
    spot_size=30,
    color_map = 'Reds',
    save="Puck_LR_Spatial_Plot.pdf"
)